This project will realize a real-time system to detect if the person showing up<br> on the webcam is wearing a mask or not, the fack mask detector model
will be<br> trained using Keras and OpenCV. The whole project will be realized with two parts:<br>
1. Using Keras in a python script to train the face detector model.
2. Using OpenCV to test results in a real-time webcam



In [ ]:
"""
1. Import essential libraries and modules to be used
"""
import cv2
import imutils
import numpy as np
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization,\
Activation, MaxPooling2D, Flatten, Dense, Dropout
from keras.models import Model, load_model
from keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from PIL import Image, ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
"""
2. Build the neural network
This network consists of two pairs od Conv and MaxPooling layers to extract features
from the image dataset, which is followed by two layers Flatten and Dropout to convert
extracted feature data into 1D vector and prevent overfitting from happening
"""
model = Sequential([
    # The 1st pair of Conv and MaxPool layers, the input shape is (300, 300, 3), which is the size of
    # the reshaped true color image from the image dataset.
    Conv2D(100, (3, 3), activation='relu', input_shape=(300, 300, 3)),
    MaxPooling2D(2, 2),
    
    # The 2nd pair of Conv and MaxPool layers
    Conv2D(100, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    Flatten(),
    Dropout(0.5),
    # Fully-connected layers
    Dense(50, activation='relu'),
    Dense(2, activation='softmax')    
])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['acc'])

In [ ]:
"""
3. Image Data Generation/Augmentation
This part would better be run only once,otherwise this will happen:
error OSError: broken data stream when reading image file
"""
TRAINING_DIR = "./train"
# Perform various data augmentation operations (e.g.,rotation, rescaling etc.)
# to extend the scale of training samples in the dataset
train_datagen = ImageDataGenerator(rescale=1.0/255,
                  rotation_range=40,
                  width_shift_range=0.2,
                  height_shift_range=0.2,
                  shear_range=0.2,
                  zoom_range=0.2,
                  horizontal_flip=True,
                  fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(TRAINING_DIR,
                           batch_size=10,
                           target_size=(300, 300))

VALIDATION_DIR = "./test"
validation_datagen = ImageDataGenerator(rescale=1.0/255)

validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR,
                                batch_size=10,
                                target_size=(300, 300))

Found 3273 images belonging to 2 classes.
Found 819 images belonging to 2 classes.


In [ ]:
"""
4. A callback checkpoint is initialized to keep saving the best model after each epoch
while training
"""
check_point = ModelCheckpoint('model2-{epoch:03d}.h5', monitor='val_loss', verbose=0,
                             save_best_only=True, mode='auto')

In [ ]:
"""
5. Train the model
"""
history = model.fit(train_generator,
           epochs = 20,
           validation_data = validation_generator,
           callbacks=[check_point])

Epoch 1/20
183/328 [===============>..............] - ETA: 42s - loss: 0.4465 - acc: 0.8044

/usr/local/lib/python3.7/dist-packages/PIL/Image.py:960: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


328/328 [==============================] - 115s 319ms/step - loss: 0.3995 - acc: 0.8307 - val_loss: 0.4214 - val_acc: 0.8242
Epoch 2/20
328/328 [==============================] - 103s 313ms/step - loss: 0.3161 - acc: 0.8784 - val_loss: 0.4784 - val_acc: 0.7924
Epoch 3/20
328/328 [==============================] - 103s 313ms/step - loss: 0.3057 - acc: 0.8769 - val_loss: 0.4595 - val_acc: 0.8107
Epoch 4/20
328/328 [==============================] - 103s 313ms/step - loss: 0.2710 - acc: 0.8937 - val_loss: 0.3487 - val_acc: 0.8632
Epoch 5/20
328/328 [==============================] - 102s 311ms/step - loss: 0.2601 - acc: 0.8928 - val_loss: 0.4612 - val_acc: 0.8230
Epoch 6/20
328/328 [==============================] - 102s 312ms/step - loss: 0.2326 - acc: 0.9102 - val_loss: 0.3347 - val_acc: 0.8730
Epoch 7/20
328/328 [==============================] - 102s 311ms/step - loss: 0.2415 - acc: 0.9004 - val_loss: 0.3570 - val_acc: 0.8755
Epoch 8/20
328/328 [==============================] - 103s 

In [ ]:
import zipfile
import os
files = zipfile.ZipFile('dataset.zip', 'r')
files.extractall(os.getcwd())